In [2]:
pip install bs4
pip install requests

SyntaxError: invalid syntax (<ipython-input-2-1dcc8df3f8f2>, line 1)

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import html

In [12]:
def scrape_wiki(wikiUrl):
        response = requests.get(wikiUrl)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        heading_tags = ["h1", "h2", "h3","h4"]
        header=[]
        for head in soup.findAll(heading_tags):
               header.append(head.text)
        for h in header:
                print(h.rstrip("\n").lstrip("\n"))
scrape_wiki('https://en.wikipedia.org/wiki/Main_Page')

Main Page
From today's featured article
Did you know ...
In the news
On this day
From today's featured list
Today's featured picture
Other areas of Wikipedia
Wikipedia's sister projects
Wikipedia languages
Navigation menu
Personal tools
Namespaces
Variants
Views
More
Search
Navigation
Contribute
Tools
Print/export
In other projects
Languages


In [45]:
def scrape_Imdb(imdbUrl):
        response = requests.get(imdbUrl)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        name=[]
        rating=[]
        year=[]
        for head in soup.findAll(class_='lister-item-content'):
            name.append(head.a.text)
        for y in soup.findAll(class_='lister-item-year text-muted unbold'):
            year.append(y.text)
        for rate in soup.findAll(class_="inline-block ratings-imdb-rating"):
            rating.append(rate.text.replace('\n','').strip())
        if(len(name)<100):
            startpage=len(name)+1
            imdbUrl2=imdbUrl.replace('start=0','start={}'.format(startpage))
            #print(imdbUrl2)
            response = requests.get(imdbUrl2)
            #print(response.status_code)
            html = response.text
            soup=BeautifulSoup(html)
            prettyhtml=soup.prettify()
            for head in soup.findAll(class_='lister-item-content'):
                name.append(head.a.text)
            for y in soup.findAll(class_='lister-item-year text-muted unbold'):
                year.append(y.text)
            for rate in soup.findAll(class_="inline-block ratings-imdb-rating"):
                rating.append(rate.text.replace('\n','').strip())
        MovieReviews=pd.DataFrame({
          "MovieName" :name,
            "Imdb Rating":rating,
            "ReleaseYear":year
        }) 
        return MovieReviews
        
df=scrape_Imdb('https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc&start=0&ref_=adv_nxt')
df.head()

,MovieName,Imdb Rating,ReleaseYear
0,The Shawshank Redemption,9.3,(1994)
1,The Godfather,9.2,(1972)
2,The Dark Knight,9.0,(2008)
3,The Godfather: Part II,9.0,(1974)
4,12 Angry Men,9.0,(1957)


In [67]:
def scrape_ImdbIndianMovies(imdbUrlI):
        response = requests.get(imdbUrlI)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        name=[]
        rating=[]
        year=[]
        for head in soup.findAll('h3',class_='lister-item-header'):
            name.append(head.a.text)
        for y in soup.findAll(class_='lister-item-year text-muted unbold'):
            year.append(y.text)
        for rate in soup.findAll('div',class_="ipl-rating-star small"):
            rating.append(rate.text.replace('\n','').strip())
        IndianMovieReviews=pd.DataFrame({
          "MovieName" :name,
           "Imdb Rating":rating,
            "ReleaseYear":year
        }) 
        return IndianMovieReviews

df=scrape_ImdbIndianMovies('https://www.imdb.com/list/ls056092300/')
print(df)

                            MovieName Imdb Rating ReleaseYear
0                     Ship of Theseus         8.1      (2012)
1                              Iruvar         8.5      (1997)
2                     Kaagaz Ke Phool           8      (1959)
3   Lagaan: Once Upon a Time in India         8.1      (2001)
4                     Pather Panchali         8.6      (1955)
..                                ...         ...         ...
95                        Apur Sansar         8.5      (1959)
96                        Kanchivaram         8.1      (2008)
97                    Monsoon Wedding         7.4      (2001)
98                              Black         8.2      (2005)
99                            Deewaar         8.1      (1975)

[100 rows x 3 columns]


In [123]:
def bookReview(bookreviewurl):
    response = requests.get(bookreviewurl)
    #print(response.status_code)
    reviewhtml = response.text
    soup=BeautifulSoup(reviewhtml)
    prettyhtml=soup.prettify()
    bookName=[]
    authorName=[]
    genre=[]
    reviewLink=[]
    reviews=[]
    BookReviews = {"BookName":[],"Author":[],"Genre":[],"Book Review":[]}
    for book in soup.select('h4'):
        bookName.append(book.a.text)
    for author in soup.findAll('p',class_='sans bold'):
        authorName.append(author.text.replace('\n','').strip())
    for g in soup.findAll(class_="genre-links hidden-phone"):
        genre.append(g.text.replace('\n','').strip())
    for link in soup.findAll(class_='read-full'):
         reviewLink.append(link.a.get('href'))
    for i in range(0,5):
        reviewUrl=bookreviewurl.replace('reviews',reviewLink[i])
        response = requests.get(reviewUrl)
        #print(reviewUrl)
        reviewshtml = response.text
        soup=BeautifulSoup(reviewshtml)
        for rev in soup.findAll('div',class_="article-body"):
            reviews.append(str(rev.text).replace('\n',''))
        BookReviews["BookName"].append(bookName[i])
        BookReviews["Author"].append(authorName[i])
        BookReviews["Genre"].append(genre[i])
        BookReviews["Book Review"].append(reviews[i])
    df=pd.DataFrame(BookReviews)
    return df

d=bookReview('https://bookpage.com/reviews')
d.head()

,BookName,Author,Genre,Book Review
0,★ Festival Days,"Jo Ann Beard, Suehyla El-Attar",Audio / Nonfiction / Essays,"Essayist Jo Ann Beard explores life, death and..."
1,Listen,"Gabi Snyder, Stephanie Graegin",Children's / Children's Picture Book,"In the opening pages of Listen, a girl stands ..."
2,★ Rise to the Sun,Leah Johnson,YA / YA Fiction,"For Olivia Brooks, the three-day Farmland Musi..."
3,Filthy Animals,Brandon Taylor,Fiction / Short Stories,"In fiction, the corporal ecstasy of sexual ten..."
4,The Woman They Could Not Silence,Kate Moore,Nonfiction / History / Women's History,We’re all familiar with the method of discredi...


In [136]:
def scrape_Cric(cricHtml):
        response = requests.get(cricHtml)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        team=[]
        matches=[]
        points=[]
        rating=[]
        for t in soup.findAll('span',class_='u-hide-phablet'):
            team.append(t.text.replace('\n','').strip())
        for m in soup.findAll('td',class_='rankings-block__banner--matches'):
            matches.append(m.text.replace('\n','').strip())
        for p in soup.findAll('td',class_='rankings-block__banner--points'):
            points.append(p.text.replace('\n','').strip())
        for r in soup.findAll('td',class_='rankings-block__banner--rating u-text-right'):
            rating.append(r.text.replace('\n','').strip())
        for m in soup.findAll('td',class_='table-body__cell u-center-text'):
            matches.append(m.text.replace('\n','').strip())
        for p in soup.findAll('td',class_='table-body__cell u-center-text'):
            points.append(p.text.replace('\n','').strip())
        for r in soup.findAll('td',class_='table-body__cell u-text-right rating'):
            rating.append(r.text.replace('\n','').strip())
        
        OdiReviews=pd.DataFrame({
          "Team" :team[:10],
            "Matches":matches[:10],
            "points":points[:10],
            "Rating":rating[:10]
        }) 
        return OdiReviews
        
df=scrape_Cric('https://www.icc-cricket.com/rankings/mens/team-rankings/odi')
df

,Team,Matches,points,Rating
0,New Zealand,17,"2,054",121
1,Australia,25,25,118
2,India,"2,945","2,945",115
3,England,29,29,115
4,South Africa,"3,344","3,344",107
5,Pakistan,27,27,97
6,Bangladesh,"3,100","3,100",90
7,West Indies,20,20,82
8,Sri Lanka,"2,137","2,137",78
9,Afghanistan,24,24,62


In [137]:
df=scrape_Cric('https://www.icc-cricket.com/rankings/womens/team-rankings/odi')
df

,Team,Matches,points,Rating
0,Australia,18,"2,955",164
1,South Africa,24,24,118
2,England,"2,828","2,828",117
3,India,17,17,111
4,New Zealand,"1,993","1,993",93
5,West Indies,20,20,85
6,Pakistan,"2,226","2,226",73
7,Bangladesh,21,21,61
8,Sri Lanka,"1,947","1,947",47
9,Ireland,12,12,13


In [155]:
def scrape_CricPlayer(cricHtml):
        response = requests.get(cricHtml)
        #print(response.status_code)
        html = response.text
        soup=BeautifulSoup(html)
        prettyhtml=soup.prettify()
        playerName=[]
        team=[]
        rating=[]
        for name in soup.findAll('div',class_='rankings-block__banner--name-large'):
            playerName.append(name.text.replace('\n','').strip())
        for t in soup.findAll('div',class_='rankings-block__banner--nationality'):
            team.append(t.text.replace('\n','').strip())
        for r in soup.findAll('td',class_='rankings-block__banner--rating'):
            rating.append(r.text.replace('\n','').strip())
        for name in soup.findAll('td',class_='table-body__cell rankings-table__name name'):
            playerName.append(name.text.replace('\n','').strip())
        for t in soup.findAll('span',class_='table-body__logo-text'):
            team.append(t.text.replace('\n','').strip())
        for r in soup.findAll('td',class_='table-body__cell rating'):
            rating.append(r.text.replace('\n','').strip())
        #print(playerName)
        
        OdiReviews=pd.DataFrame({
          "playerName" :playerName[:10],
            "team":team[:10],
            "Rating":rating[:10]
        }) 
        return OdiReviews
        
bowlingMendf=scrape_CricPlayer('https://www.icc-cricket.com/rankings/mens/player-rankings/odi/bowling')
battingMendf=scrape_CricPlayer('https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting')
battingMendf


,playerName,team,Rating
0,Babar Azam,PAK,857
1,Virat Kohli,IND,825
2,Rohit Sharma,IND,801
3,Ross Taylor,NZ,791
4,Aaron Finch,AUS,785
5,Jonny Bairstow,ENG,778
6,Fakhar Zaman,PAK,778
7,Francois du Plessis,SA,773
8,David Warner,AUS,773
9,Shai Hope,WI,756


In [152]:
bowlingMendf

,playerName,team,Rating
0,Trent Boult,NZ,713
1,Mehedi Hasan,BAN,708
2,Mujeeb Ur Rahman,AFG,691
3,Matt Henry,NZ,690
4,Jasprit Bumrah,IND,666
5,Kagiso Rabada,SA,665
6,Chris Woakes,ENG,660
7,Josh Hazlewood,AUS,646
8,Pat Cummins,AUS,645
9,Mustafizur Rahman,BAN,638


In [156]:
bowlingWoMendf=scrape_CricPlayer('https://www.icc-cricket.com/rankings/womens/player-rankings/odi/bowling')
battingWoMendf=scrape_CricPlayer('https://www.icc-cricket.com/rankings/womens/player-rankings/odi/batting')

In [157]:
bowlingWoMendf

,playerName,team,Rating
0,Jess Jonassen,AUS,762
1,Megan Schutt,AUS,747
2,Marizanne Kapp,SA,717
3,Shabnim Ismail,SA,681
4,Jhulan Goswami,IND,655
5,Katherine Brunt,ENG,641
6,Poonam Yadav,IND,638
7,Ayabonga Khaka,SA,616
8,Ellyse Perry,AUS,610
9,Shikha Pandey,IND,607


In [158]:
battingWoMendf

,playerName,team,Rating
0,Tammy Beaumont,ENG,758
1,Lizelle Lee,SA,756
2,Alyssa Healy,AUS,746
3,Stafanie Taylor,WI,723
4,Meg Lanning,AUS,715
5,Amy Satterthwaite,NZ,710
6,Smriti Mandhana,IND,709
7,Mithali Raj,IND,685
8,Natalie Sciver,ENG,683
9,Laura Wolvaardt,SA,679


In [203]:
def mob_data(amazonHtml):
    response = requests.get(amazonHtml)
    #print(response.status_code)
    mobileHtml = response.text
    soup=BeautifulSoup(mobileHtml)
    mobileName=[]
    price=[]
    imageUrl=[]
    avgRating=[]
    for mob in soup.findAll('h2',class_='a-size-mini a-spacing-none a-color-base s-line-clamp-2'):
        mobileName.append(mob.text.replace('\n','').strip())
    for p in soup.findAll('span',class_='a-offscreen'):
        price.append(p.text.replace('\n','').strip())
    for i in soup.findAll('img',class_='s-image'):
        imageUrl.append(i['src'])
    for ar in soup.findAll('span',class_='a-icon-alt'):
        avgRating.append(ar.text)
    
    Mobiledata=pd.DataFrame({
          "mobileName" :mobileName[:10],
         "price":price[:10],
        "imageUrl":imageUrl[:10],
          "rating": avgRating[:10]
    }) 
    return Mobiledata

df=mob_data('https://www.amazon.in/s?k=mobiles+under+20000&rh=n%3A1389432031&dc&ref=a9_asc_1')
df

,mobileName,price,imageUrl,rating
0,"Samsung Galaxy M12 (Blue,4GB RAM, 64GB Storage...","₹10,999",https://m.media-amazon.com/images/I/71yYaNztZ0...,4.1 out of 5 stars
1,"Redmi 9A (Nature Green, 2GB RAM, 32GB Storage)...","₹12,999",https://m.media-amazon.com/images/I/71sxlhYhKW...,4.2 out of 5 stars
2,"Samsung Galaxy M11 (Metallic Blue, 4GB RAM, 64...","₹6,799",https://m.media-amazon.com/images/I/71GQUxuSpn...,4.2 out of 5 stars
3,"Oppo A31 (Mystery Black, 6GB RAM, 128GB Storag...","₹8,499",https://m.media-amazon.com/images/I/71KCwNV6Mu...,4.2 out of 5 stars
4,"TECNO Spark 7T(Jewel Blue, 4GB RAM, 64GB Stora...","₹9,499",https://m.media-amazon.com/images/I/81aWyRY67S...,4.1 out of 5 stars
5,"Redmi 9 (Sky Blue, 4GB RAM, 64GB Storage)| 500...","₹14,999",https://m.media-amazon.com/images/I/71A9Vo1Bat...,4.2 out of 5 stars
6,"Redmi 9 Power (Mighty Black, 6GB RAM, 128GB St...","₹8,999",https://m.media-amazon.com/images/I/61LHaUOheh...,4.2 out of 5 stars
7,"Vivo Y51A (Titanium Sapphire, 8GB, 128GB Stora...","₹10,999",https://m.media-amazon.com/images/I/71BRHXBME2...,4.1 out of 5 stars
8,"Samsung Galaxy M31 (Ocean Blue, 8GB RAM, 128GB...","₹8,799",https://m.media-amazon.com/images/I/71-Su4Wr0H...,4.3 out of 5 stars
9,"Redmi 9 (Carbon Black, 4GB RAM, 64GB Storage) ...","₹10,999",https://m.media-amazon.com/images/I/716nHhG9SW...,4.2 out of 5 stars


In [217]:
def weather(weatherURl):
    response = requests.get(weatherURl)
    #print(response.status_code)
    weatherHtml = response.text
    soup=BeautifulSoup(weatherHtml)
    period=[]
    short_des=[]
    temperature=[]
    description=[]
    #divs=soup.findAll('div',class_='tombstone-container')
    temptag=['temp temp-low','temp temp-high']     
    for time in soup.findAll('p',class_='period-name'):
        period.append(time.text.replace('\n','').strip())
    for short in soup.findAll('p',class_='short-desc'):
        short_des.append(short.text.replace('\n','').strip())
    for t in soup.findAll('p',class_=temptag):
        temperature.append(t.text.replace('\n','').strip())
    for desc in soup.findAll('div',class_='col-sm-10 forecast-text'):
        description.append(desc.text.replace('\n','').strip())
    
    Forecast=pd.DataFrame({
          "period" :period[:7],
         "short description":short_des[:7],
        "temperature":temperature[:7],
          "description": description[:7]
    }) 
    return Forecast

df=weather('https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.YNYm1xHitPY')
df

,period,short description,temperature,description
0,ThisAfternoon,Sunny andBreezy,High: 65 °F,"Sunny, with a steady temperature around 65. Br..."
1,Tonight,IncreasingClouds andBreezy thenCloudy,Low: 57 °F,"Increasing clouds, with a low around 57. Breez..."
2,Saturday,Partly Sunnythen Sunnyand Breezy,High: 70 °F,"Mostly cloudy through mid morning, then gradua..."
3,SaturdayNight,IncreasingClouds andBreezy thenMostly Cloudy,Low: 58 °F,"Increasing clouds, with a low around 58. Breez..."
4,Sunday,Partly Sunnythen MostlySunny andBreezy,High: 70 °F,"Mostly sunny, with a high near 70. Breezy, wit..."
5,SundayNight,Partly Cloudyand Breezythen MostlyCloudy,Low: 58 °F,"Mostly cloudy, with a low around 58. Breezy."
6,Monday,Mostly Sunnyand Breezy,High: 71 °F,"Mostly sunny, with a high near 71. Breezy."


In [220]:
def job_details(jobUrl):
    response = requests.get(jobUrl)
    #print(response.status_code)
    jobHtml = response.text
    soup=BeautifulSoup(jobHtml)
    #prettyhtml=soup.prettify()
    jobTitle=[]
    CompanyName=[]
    ctc=[]
    ApplyDate=[]
    for job in soup.findAll('div',class_='heading_4_5 profile'):
        jobTitle.append(job.a.text.replace('\n','').strip())
    for company in soup.findAll('div',class_='heading_6 company_name'):
        CompanyName.append(company.a.text.replace('\n','').strip())
    for c in soup.findAll(class_='item_body'):
        if (c.text.replace('\n','').strip().endswith('LPA')):
            ctc.append(c.text.replace('\n','').strip())  
        if (c.text.replace('\n','').strip().endswith('21')):
            ApplyDate.append(c.text.replace('\n','').strip())  

        
    df=pd.DataFrame({
        "jobTitle":jobTitle,
        "Company Name":CompanyName,
        "ctc":ctc,
        "Apply Date":ApplyDate
    })
    return df

d=job_details('https://internshala.com/fresher-jobs')
d

,jobTitle,Company Name,ctc,Apply Date
0,Salesforce Developer,Softsensor.ai,3 - 5 LPA,25 Jul' 21
1,Full Stack Developer,ProtonAutoML,3 - 4 LPA,25 Jul' 21
2,Business Development Specialist,ProtonAutoML,3 - 3.5 LPA,25 Jul' 21
3,Associate Software Developer,RoaDo,4 - 6 LPA,25 Jul' 21
4,Associate UI/UX Designer,RavGins International Private Limited (Wobb.ai),3 - 4 LPA,25 Jul' 21
5,Business Development Associate (Sales),Global Sun,3 - 4.8 LPA,25 Jul' 21
6,Receptionist And Billing Manager,Touchstone Partners,3 - 7 LPA,25 Jul' 21
7,Node.js Backend Developer (AWS/GCP/API),Labellerr By Tensor Matics Private Limited,7 - 10 LPA,25 Jul' 21
8,Business Development Executive,UFaber Edutech,3 - 5 LPA,25 Jul' 21
9,Software Developer,Aptigenz Solutions Private Limited,3 - 3.2 LPA,24 Jul' 21


In [4]:
def house_details(Url):
    response = requests.get(Url)
    #print(response.status_code)
    Html = response.text
    soup=BeautifulSoup(Html)
    #prettyhtml=soup.prettify()
    houseTitle=[]
    location=[]
    Area=[]
    EMI=[]
    Price=[]
    for house in soup.findAll('h2',class_='heading-6 font-semi-bold nb__1AShY'):
        houseTitle.append(house.text.replace('\n','').strip())
    for loc in soup.findAll('div',class_='nb__2CMjv'):
        location.append(loc.text.replace('\n','').strip())
    for p in soup.findAll('div',class_='font-semi-bold heading-6'):
        for k in p.findAll('span'):
            if (k.text.replace('\n','').strip('â\x82¹')!=''):
                Price.append(k.text.replace('\n','').strip('â\x82¹'))
    for e in soup.findAll('div',class_='font-semi-bold heading-6',id="roomType"):
        EMI.append(e.text.replace('\n','').strip('â\x82¹'))
    for a in soup.findAll('div',class_='nb__3oNyC'):
        Area.append(a.text.replace('\n','').strip())
            
    df=pd.DataFrame({
        "House Title":houseTitle,
        "location":location,
        "Area":Area,
        "EMI":EMI,
        "Price":Price
    })
    return df

d=house_details('https://www.nobroker.in/property/sale/mumbai/multiple?searchParam=W3sibGF0IjoxOS4xMTc4NTQ4LCJsb24iOjcyLjg2MzEzMDQsInBsYWNlSWQiOiJDaElKTWJIZlFSdTI1enNSTWF6ZFkzVXBhS1kiLCJwbGFjZU5hbWUiOiJBbmRoZXJpIEVhc3QifSx7ImxhdCI6MTkuMTM2NDAxNiwibG9uIjo3Mi44Mjk2MjUyLCJwbGFjZUlkIjoiQ2hJSjNaSFl0aGkyNXpzUk10RXh4MWVvLUpFIiwicGxhY2VOYW1lIjoiQW5kaGVyaSBXZXN0In1d&radius=2.0&type=RK1&propertyAge=0')
d

,House Title,location,Area,EMI,Price
0,1 RK Flat For Sale In Arpit Building In Andh...,"Marol MIDC Industry Estate,beside akruti trade...",250 sqft,"33,242/Month",58 Lacs
1,1 RK Apartment For Sale In Crescent Landmark...,"Crescent LandmarkÂ Makwana road, gamdevi, nea...",450 sqft,"57,314/Month",1 Crore
2,1 RK Flat For Sale In Taruvel Chs In Andheri...,J B Nagar,400 sqft,"54,448/Month",95 Lacs
3,"1 RK Flat For Sale In Saicharan Chs,sra In A...",Sai Wadi near Hubtown Sunmist,280 sqft,"36,108/Month",63 Lacs
4,1 RK Flat For Sale In Rajendra Co-operative ...,"Tarun Bharat Area,Chakala, Near Oriental Aster",460 sqft,"54,448/Month",95 Lacs
5,1 RK In Independent House For Sale In Andher...,"Independent House, s.v.p nagar opp st mary hig...",250 sqft,"51,583/Month",90 Lacs
6,1 RK In Independent House For Sale In Andher...,Anand Nagar Marol pipe Line,200 sqft,"22,925/Month",40 Lacs
7,1 RK Flat For Sale In Shraddha Heights In An...,Shraddha Heights Teli Galli Cross Rd Sampada S...,300 sqft,"57,314/Month",1 Crore
8,1 RK Flat For Sale In New Sangam Society In ...,new versova link road,290 sqft,"26,937/Month",47 Lacs
9,1 RK Flat For Sale In Hill Rock Society In A...,"Veera Desai Road, Near M V M?s College of Comm...",290 sqft,"34,961/Month",61 Lacs
